# LCEL(LangChain Expression Language)
https://reference.langchain.com/python/langchain_core/runnables/

https://reference.langchain.com/python/langchain_core/runnables/?h=runnablelambd#langchain_core.runnables.base.RunnableLambda
  
- LCEL(LangChain Expression Language)은 LangChain에서 체인을 선언적으로 구성할 수 있게 해주는 도메인 특화 언어다.  
- `|` 연산자를 사용해 프롬프트, 모델, 파서 등을 파이프라인처럼 연결한다.

**주요 특징**

- **선언적 문법**: Unix 파이프처럼 `chain = prompt | model | parser` 형태로 직관적이다.  
- **모듈성·유연성**: 프롬프트, LLM, 파서, 검색기, 메모리 등 컴포넌트를 자유롭게 조합할 수 있다.  
- **동기/비동기 지원**: 단일 코드로 동기식·비동기식 실행을 모두 처리할 수 있다.  
- **병렬 처리 최적화**: 병렬 실행 가능한 단계는 자동으로 병렬화해 지연 시간을 줄인다.  
- **고급 기능 기본 제공**:  
  - 스트리밍 출력으로 응답 속도를 향상시킨다.  
  - 실패 시 재시도와 폴백 경로를 설정할 수 있다.  
  - 중간 결과에 접근해 디버깅이나 진행 상황 표시가 가능하다.

**LCEL의 주요 기능**

1. **스트리밍 지원**: 첫 토큰 도달 시간을 단축해 실시간성을 높인다.  
2. **비동기 지원**: asyncio 환경 등 다양한 실행 환경을 동일 코드로 지원한다.  
3. **병렬 실행 최적화**: 병렬화 가능한 단계는 자동으로 분리해 동시에 실행한다.  
4. **재시도·폴백 구성**: 오류 발생 시 지정 횟수만큼 재시도하거나 대체 경로를 실행한다.  
5. **중간 결과 접근**: 최종 출력 이전에 각 단계의 출력을 확인할 수 있다.

**기본 구성 요소**

- **Runnable**: LCEL의 모든 컴포넌트가 상속하는 기본 클래스다.  
- **Chain**: 여러 Runnable을 순차적으로 실행한다.  
- **RunnableMap**: 여러 Runnable을 병렬로 실행한다.  
- **RunnableSequence**: Runnable들의 시퀀스를 정의한다.  
- **RunnableLambda**: 파이썬 함수를 래핑해 Runnable로 만든다.

In [2]:
%pip install langchain langchain-openai -Uqqq

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv  # .env 파일의 환경변수 로드
import os                       # 환경변수 접근용

load_dotenv()                   # 현재 위치의 .env를 읽어와 환경변수로 등록
os.environ["OPENAI_API_KEY"] = os.getenv("openai_key")  # .env의 openai_key 값을 OPENAI_API_KEY로 등록
os.environ["LANGSMITH_TRACING"] = 'true'                # LangSmith 트레이싱 활성화
os.environ["LANGSMITH_ENDPOINT"] = 'https://api.smith.langchain.com'  # LangSmith API 엔드포인트 설정
os.environ["LANGSMITH_PROJECT"] = 'skn23-langchain'                   # LangSmith 프로젝트명 설정
os.environ["LANGSMITH_API_KEY"] = os.getenv("langsmith_key")          # .env의 langsmith_key 값을 LANGSMITH_API_KEY로 등록

## RunnableLambda
일반 python 함수를 lcel 체인에서 사용할 수 있게 wrapping 처리하는 클래스

In [ ]:
from langchain_core.runnables import RunnableLambda  # 입력을 받아 함수를 실행하는 Runnable

runnable = RunnableLambda(lambda x: len(x))          # 입력 x의 길이를 반환하는 Runnable 생성
runnable.invoke('플레이데이터 독산 skn-23 화이팅!')

21

In [ ]:
# batch : 여러 건의 입력을 일괄처리하는 메소드
runnable.batch(['플레이데이터', '독산', 'skn-23', '화이팅!'])

[6, 2, 6, 4]

In [ ]:
# 섭씨 입력값을 화씨로 변환하는 runnable 생성
def celsius_to_fahrenheit(celsius):
    """섭씨 온도를 화씨 온도로 변환하는 함수"""
    return celsius * 9 / 5 + 32  # 섭씨 -> 화씨 변환 공식

celsius_temps = [0, 25, 100, -10, 37]
runnable = RunnableLambda(celsius_to_fahrenheit)  # 함수를 Runnable로 래핑
runnable.batch(celsius_temps)  # 여러 입력을 한 번에 변환

[32.0, 77.0, 212.0, 14.0, 98.6]

In [9]:
# stream으로 제너레이터 출력 스트리밍하기
import time  # 출력 딜레이용

def gen(x):
    """입력 문자열을 한 글자(문자)씩 yield하는 제너레이터"""
    for y in x:    # 입력을 순회(문자 단위)
        yield y    # 한 글자씩 반환(스트리밍 단위)

runnable = RunnableLambda(gen)  # 문자열 제너레이터 함수를 Runnable로 래핑
for chunk in runnable.stream("안녕하세요? 안녕하세요! 안녕하세요~ 안녕하세요? 안녕하세요! 안녕하세요~ 안녕하세요? 안녕하세요! 안녕하세요~ 안녕하세요? 안녕하세요! 안녕하세요~ 안녕하세요? 안녕하세요! 안녕하세요~"):
    print(chunk, end='', flush=True)  # chunk를 줄바꿈 없이 즉시 출력
    time.sleep(0.1)                   # 0.1초 간격으로 천천히 출력

안녕하세요? 안녕하세요! 안녕하세요~ 안녕하세요? 안녕하세요! 안녕하세요~ 안녕하세요? 안녕하세요! 안녕하세요~ 안녕하세요? 안녕하세요! 안녕하세요~ 안녕하세요? 안녕하세요! 안녕하세요~

In [ ]:
def gen(x):
    """iterable을 받아 원소를 하나씩 yield하는 제너레이터"""
    for y in x:    # 입력을 순회(iterable)
        yield y    # 원소를 하나씩 순회

gen10 = gen(range(10))  # 0~9를 하나씩 꺼내는 제너레이터 생성
gen10  # 객체 자체 실행 (실행 전 / 소모 전)

<generator object gen at 0x00000242794BB040>

In [ ]:
for n in gen10:  # gen10에서 값을 하나씩 꺼내며 순회 (꺼낸 값은 다시 사용못함)
    print(n)

In [25]:
gen10 = gen(range(10))  # 0~9를 하나씩 꺼내는 제너레이터 생성

In [26]:
next(gen10)  # 제너레이터에서 다음 값을 1개 반환 (다 꺼내면 StopIteration 발생)

0

## RunnableSequence
Runnable객체를 순차연결하는 Runnable 객체

In [ ]:
from langchain_core.runnables import RunnableSequence  # Runnable들을 순서대로 연결하는 시퀀스

runnable1 = RunnableLambda(lambda x: {'foo': x})  # 입력 x를 {'foo': x} 형태로 변환
runnable2 = RunnableLambda(lambda x: [x] * 3)     # 입력 x를 리스트 3번 반복 [x, x, x]

chain = RunnableSequence(runnable1, runnable2)
# chain = runnable1 | runnable2  # 위와 동일
chain.invoke(3)

[{'foo': 3}, {'foo': 3}, {'foo': 3}]

## RunnableParellel
여러 Runnable객체를 인자로 받아, 병렬처리 후 각각의 응답을 하나의 dict 형태로 반환

In [30]:
from langchain_core.runnables import RunnableParallel  # 여러 Runnable들을 같은 입력으로 병렬 실행

runnable1 = RunnableLambda(lambda x: {'foo': x})  # 입력 x를 {'foo': x} 형태로 변환
runnable2 = RunnableLambda(lambda x: [x] * 3)     # 입력 x를 리스트 3번 반복 [x, x, x]

chain = RunnableParallel(r1=runnable1, r2=runnable2)  # r1, r2를 병렬로 실행해 결과를 dict로 묶음
chain.invoke(3)

{'r1': {'foo': 3}, 'r2': [3, 3, 3]}

사용자가 지정한 주제(topic)에 대해서 삼행시, 농담, 시를 각각 생성하여 하나의 응답을 작성한다.

In [34]:
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnableLambda  # 병렬 실행 / 함수 Runnable

llm = init_chat_model('openai:gpt-5.2')
output_parser = StrOutputParser()

acrostic_poem_prompt = PromptTemplate.from_template(
    '당신은 창의적인 n행시 고수입니다. 다음 주제로 n행시를 지어주세요.\n\n주제:{topic}'
)
acrostic_poem_chain = acrostic_poem_prompt | llm | output_parser

joke_prompt = PromptTemplate.from_template(
    '당신은 너무 웃긴 농담 고수입니다. 다음 주제로 허를 찌르는 농담을 하나 해주세요.\n\n주제:{topic}'
)
joke_chain = joke_prompt | llm | output_parser

poem_prompt = PromptTemplate.from_template(
    '당신은 위대한 시인입니다. 다음 주제로 감성적인 시를 지어주세요.\n\n주제:{topic}'
)
poem_chain = poem_prompt | llm | output_parser

chain = RunnableParallel(
    acrostic_poem = acrostic_poem_chain,
    joke = joke_chain,
    poem = poem_chain
)

def combine_result(input_dict: dict) -> str:
    """병렬 결과(n행시/농담/시)를 한 문자열로 합쳐 반환한다."""
    acrostic_poem = input_dict['acrostic_poem']
    joke = input_dict['joke']
    poem = input_dict['poem']
    return f"""
n행시:
{acrostic_poem}

농담:
{joke}

시:
{poem}
"""

chain = chain | RunnableLambda(combine_result)
print(chain.invoke({'topic': '아이스크림'}))


n행시:
아: 아주 더운 날엔  
이: 이대로 녹기 전에  
스: 스르르 입안에 넣어  
크: 크리미한 달콤함을 느끼고  
림: 림(임)자, 오늘 하루도 기분 좋게 마무리!

농담:
아이스크림 먹다가 너무 감동해서 눈물이 났어요.  
왜냐면… **내 인생도 얘처럼 빨리 녹거든요.**

시:
혀끝에 닿는 순간  
여름은 잠깐,  
세상에서 가장 부드러운 속도로 녹아내린다.

유리창 밖의 햇빛이  
마치 오래된 약속처럼 반짝이면  
나는 두 손으로 작은 컵을 감싸 쥔다—  
차가움이 손바닥을 타고 올라와  
마음의 뜨거운 곳을 살짝 식혀준다.

바닐라의 하얀 숨,  
초콜릿의 깊은 그림자,  
딸기의 붉은 웃음이  
한 숟갈 안에서 서로를 안아  
어린 날의 내 이름을 불러낸다.

녹아 흐르는 건 아이스크림만이 아니다.  
미처 말하지 못한 미안함,  
돌아갈 수 없는 오후,  
괜찮은 척하던 서운함이  
조용히, 아주 조용히 풀린다.

그래서 나는  
다 녹기 전에 서둘러 먹으면서도  
어쩐지 다 녹아버리길 바란다.  

사라지는 것만이  
이렇게 다정할 때가 있으니까.



## RunnablePassThought
- 사용자의 입력값을 그대로 전달
- 입력 dict를 확장

In [36]:
from langchain_core.runnables import RunnablePassthrough  # 입력을 그대로 통과시키는 Runnable

llm = init_chat_model('openai:gpt-4.1-mini')
output_parser = StrOutputParser()

prompt = PromptTemplate.from_template(
    '당신은 창의적인 n행시 고수입니다. 다음 주제로 n행시를 지어주세요.\n\n주제:{topic}'
)

chain = (
    {'topic': RunnablePassthrough()}  # 입력(문자열)을 그대로 받아 {'topic': 입력}으로 매핑
    | prompt
    | llm
    | output_parser
)

chain.invoke('텀블러')

'텀: 텀텀 뛰는 일상 속  \n블: 블렌딩된 나만의 향기와  \n러: 러블리하게 하루를 채워가요!'

In [37]:
from langchain_core.runnables import RunnablePassthrough  # 입력을 그대로 통과시키는 Runnable

prompt = PromptTemplate.from_template("""
당신은 창의적인 {n}행시 고수입니다. 다음 주제로 {n}행시를 지어주세요.

# 주제:
{topic}

# 출력형식:
===== <주제> <n>행시 =====
<n행시 작성>
"""
)

chain = (
    {'topic': RunnablePassthrough()}
    | RunnablePassthrough.assign(     # 기존 dict를 확장해서 새 key를 추가
        n=lambda x: len(x['topic'])   # topic 길이로 n 계산
    )
    | prompt
    | llm
    | output_parser
)

chain.invoke('학원')

'===== 학원 2행시 =====  \n학 - 학교보다 더 가까운 곳에서  \n원 - 원하는 꿈을 키워 가네'